In [1]:
#cpu 사용
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 344407625041552242
xla_global_id: -1
]


# Textcuboid


## 1) import Reuters data

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import reuters

In [4]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(test_split=0.2)

In [5]:
print('훈련용 뉴스 기사 : {}'.format(len(train_data)))
print('테스트용 뉴스 기사 : {}'.format(len(test_data)))
num_classes = len(set(train_labels))
print('카테고리 : {}'.format(num_classes))

훈련용 뉴스 기사 : 8982
테스트용 뉴스 기사 : 2246
카테고리 : 46


In [26]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

In [27]:
word_to_index = reuters.get_word_index()
print(word_to_index)
# word_to_index는 올바른 indexing인데 정작 index_to_word는 +3한 index를 사용한다.

{'mdbl': 10996, 'fawc': 16260, 'degussa': 12089, 'woods': 8803, 'hanging': 13796, 'localized': 20672, 'sation': 20673, 'chanthaburi': 20675, 'refunding': 10997, 'hermann': 8804, 'passsengers': 20676, 'stipulate': 20677, 'heublein': 8352, 'screaming': 20713, 'tcby': 16261, 'four': 185, 'grains': 1642, 'broiler': 20680, 'wooden': 12090, 'wednesday': 1220, 'highveld': 13797, 'duffour': 7593, '0053': 20681, 'elections': 3914, '270': 2563, '271': 3551, '272': 5113, '273': 3552, '274': 3400, 'rudman': 7975, '276': 3401, '277': 3478, '278': 3632, '279': 4309, 'dormancy': 9381, 'errors': 7247, 'deferred': 3086, 'sptnd': 20683, 'cooking': 8805, 'stratabit': 20684, 'designing': 16262, 'metalurgicos': 20685, 'databank': 13798, '300er': 20686, 'shocks': 20687, 'nawg': 7972, 'tnta': 20688, 'perforations': 20689, 'affiliates': 2891, '27p': 20690, 'ching': 16263, 'china': 595, 'wagyu': 16264, 'affiliated': 3189, 'chino': 16265, 'chinh': 16266, 'slickline': 20692, 'doldrums': 13799, 'kids': 12092, 'cl

In [28]:
len(word_to_index)

30979

In [29]:
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key # pad, sos, unk, NULL 포함

In [30]:
for index, token in enumerate(("<pad>", "<sos>", "<unk>")): # index=3은 NULL
    index_to_word[index] = token

In [34]:
#문장 추출
to_txt=[]
for i in range(8982):
  
    to_txt.append(' '.join([index_to_word[index] for index in train_data[i]])[6:])
    
for i in range(2246):
    to_txt.append(' '.join([index_to_word[index] for index in test_data[i]])[6:])

In [35]:
len(to_txt)

11228

In [36]:
to_txt

['mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3',
 "generale de banque sa lt genb br and lt heller overseas corp of chicago have each taken 50 pct stakes in factoring company sa belgo factors generale de banque said in a statement it gave no financial details of the transaction sa belgo factors' turnover in 1986 was 17 5 billion belgian francs reuter 3",
 'shr 3 28 dlrs vs 22 cts shr diluted 2 99 dlrs vs 22 cts net 46 0 mln vs 3 328 000 avg shrs 14 0 mln vs 15 2 mln year shr 5 41 dlrs vs 1 56 dlrs shr diluted 4 94 dlrs vs 1 50 dlrs net 78 2 mln vs 25 9 mln avg shrs 14 5 mln vs 15 1 mln note earnings per share reflect the two for one spli

In [38]:
#문장 끝에 온점 추가
for i in range(len(to_txt)):
    to_txt[i]=to_txt[i] + " ."

In [41]:
import numpy as np

train_data=np.array(to_txt)

In [44]:
import os
if not os.path.exists("swb/train"):
    os.makedirs("swb/train")
for i in range(0,train_data.shape[0],6):
    text = "\n".join(train_data[i:i+6].tolist())
    fp = open("swb/train/"+str(i)+".txt","w")
    fp.write(text)
    fp.close()

In [45]:
#소문자
from collections import Counter

texts = " ".join(train_data.tolist())
words = texts.split(" ")
print("Number of tokens in Training data = ",len(words))
dictionary = Counter(words)
print("Size of Vocab",len(dictionary))
sorted_vocab = ["<S>","</S>","<UNK>"]
sorted_vocab.extend([pair[0] for pair in dictionary.most_common()])
 
text = "\n".join(sorted_vocab)
fp = open("swb/vocab.txt","w")
fp.write(text)
fp.close()

Number of tokens in Training data =  1638886
Size of Vocab 30980


In [1]:
#하이퍼 파라미터 옵션(학습이 끝난 뒤엔 꼭 n_character=262로 해줘야 하므로 재설정. 기본값이 261로 돼있을 것임)
options = {
     'bidirectional': True,

     'char_cnn': {'activation': 'relu',
      'embedding': {'dim': 16},
      'filters': [[1, 32],
       [2, 32],
       [3, 64],
       [4, 128],
       [5, 256],
       [6, 512],
       [7, 1024]],
      'max_characters_per_token': 50,
      'n_characters': 262,
      'n_highway': 2},
    
     'dropout': 0.1,
    
     'lstm': {
      'cell_clip': 3,
      'dim': 4096,
      'n_layers': 2,
      'proj_clip': 3,
      'projection_dim': 512,
      'use_skip_connections': True},
    
     'all_clip_norm_val': 10.0,
    
     'n_epochs': 10,
     'n_train_tokens': 1638886,
     'batch_size': 64,
     'n_tokens_vocab': 30980,
     'unroll_steps': 20,
     'n_negative_samples_batch': 8192,
    }

In [3]:
import json
file_path = "./swb/checkpoint/options.json"
with open(file_path, 'w') as outfile:
    json.dump(options, outfile, indent=4)